# BMEB W4020: Circuits in the Brain 2023Fall Homework \#3 Handout
*Author:* Shashwat Shukla <shashwat.shukla@columbia.edu> Bruce Yi Bu <yb2520@columbia.edu>


*Based on Previous Work by:* Tingkai Liu, Mehmet Kerem Turkcan, Chung-Heng Yeh,
                  Konstantinos Psychas, Cong Han

Copyright 2012-2023 Shashwat Shukla, Bruce Yi Bu, Tingkai Liu and Aurel A. Lazar


# PROBLEM \#1 - $\delta$-insensitive TDM

The stimulus of a single-input single-output (SISO) TEM is modeled as
a bandlimited function of the form
$$
u(t)= \sum_{k=1}^{15} u(kT) \frac{\sin \Omega (t-kT)}{\Omega (t-kT)},
$$
where $\Omega = 2 \pi \cdot 25$ Hz and $T=\frac{\pi}{\Omega}$.

Assume that the TEM describes a IAF neuron. After choosing appropriate TEM parameter values (provided in the skeleton) generate the trigger times in the time interval $[-2T, 18T]$.

### Tasks
1. Implement a threshold insensitive ($\delta$-insensitive) decoding algorithm for IAF.
2. Plot the recovery error (difference between the input stimulus and the recovered waveform) and calculate the whole-signal Signal-to-Noise ratio of the recovery in decibel (dB); plot SNR as function of time.
3. Compare the $\delta$-insensitive recovery result with the $\delta$-sensitive result (implementation of $\delta$-sensitive algorithm can be found in lecture notebook).


### Note
* The derivation of $\delta$-insensitive TDM decoding is available in **Chapter 6**. Highly recommend reading through the entire chapter thoroughly as it will **massively** help with the following chapters and lectures; pay attention to the *Compensation Principle*.

* Whole-signal SNR of the recovery can be calculated for signal $u(t)$ and recovered signal $u_{rec}(t)$ as 
$
SNR = 10\log_{10}\left(\frac{mean(u^2)}{mean((u-u_{rec})^2)}\right)
$
the result will be in dB
* To plot SNR as a function of time, use 
$
SNR = 10\log_{10}\left(\frac{u^2}{(u-u_{rec})^2}\right)
$ to preserve the time dimension.

# Problem 2 - Derivation of TEM/TDM algorithm for RHH Neuron with dendritic processing
In this problem you are asked to derive and implement a TEM/TDM algorithm for the reduced Hodgkin-Huxley neuron (BSG in the figure below) with dendritic processing modeled as a linear filter ($h(t)$ in the figure below).

The encoding circuit consisting of a filter in cascade with a RHH neuron:
   <center><img src="./filter-bsg.png" width=650/></center>

In the figure above, the external current injected into the neuron is given as 
$$
I_{ext}(t) = b + \underbrace{\left( u \ast h \right)(t)}_{v(t)}
$$
where $b$ is the injected current (also written as $I$ in other texts).


### Tasks
1. Generate the impulse response of $h(t)$ and visualize.
2. With $b=20$, encode a randomly generated input stimulus (from Problem 1) using a reduced PIF neuron that is equivalent to the HH neuron model. 
3. Derive an algorithm to recover the signal $u(t)$ from the recieved spikes.
    1. Writing down the $t$-transform of the encoding circuit shown above in an inner product form.
    2. Find the time decoding machine (TDM) that recovers the signal $u$. Particularly, provide forms for $q_k$ and $[G]_{lk}$. Please write down the important procedures.
3. Recover the signal $u(t)$ from output spike times of the reduced PIF and show encoding error and SNR (as in Problem \#1 above, calculate and plot).


### Stimulus and Filter
Use the stimulus $u(t)$ from Problem \#1, and the filter $h$ is 
$$
h(t)= 3 \cdot 150 \cdot \mbox{exp}\left(-100 t\right)\left(\frac{(150 t)^3}{3!}-\frac{(150t)^5}{5!}\right) \cdot \mathbb{1}_{t\ge 0}
$$
note that $\mathbb{1}_{t\ge 0}$ is also known as the Heaviside Step function which ensures that the filter $h(t)$ is causal.

### Note
You know the filter $h$ and filtered output $v(t) = (u \ast h)(t)$, but you _do not_ know $u(t)$. You can read the spike times, and you want to recover $u(t)$ from the spikes.

# Problem 1

In [ ]:
%matplotlib inline
import numpy as np
np.random.seed(0)  # fix random seed
import matplotlib.pyplot as plt

import typing as tp
from scipy import signal
from scipy.linalg import circulant
from scipy.integrate import cumulative_trapezoid as cumtrapz
from compneuro.utils.phase_response import PIF, iPRC
from compneuro.utils.neuron import limit_cycle
from compneuro.utils.signal import spike_detect, spike_detect_local
from compneuro.neurons.hodgkin_huxley_3state import HodgkinHuxley3State
from compneuro.utils.signal import convolve

plt.rcParams["figure.figsize"] = [10, 5]
plt.rcParams["figure.dpi"] = 100

In [ ]:
# TODO: the stimulus ut
def ut(t,T, omega):
    return np.nan

### Part 1. Implement a threshold insensitive ($\delta$-insensitive) decoding algorithm for IAF.

In [ ]:
# TODO: implement IAF
iaf_params = dict(
    kappa = 1# mF
    delta = 0.01,  # mV
    b = 1 #mu A
    omega = omega
)

def integrate_step_IAF():
    return np.nan

def iaf_encode():
    return np.nan

In [ ]:
# TODO: endode with IAF
dt = 1e-6
# T = np.pi/omega
# t = np.arange(-2*T, 18*T, dt)
# u = ut(t,T,omega)
# tk = []

**Disclaimer** implementation of $\delta$-sensitive algorithm can be found in lecture notebook.

In [ ]:
# TODO: Recovery 

# functions for computing the matrices, you can (and probably should) add more
def compute_G(t, tk):
#     return G
def compute_q(t, tk):
#     return q


def iaf_decoder_sen(t, tk):
    # TODO: find the sensitive case
    return np.nan

def iaf_decoder_insen(t, tk):
    q = compute_q(t, tk)
    G = compute_G(t, tk)
    # TODO: implement the insensitive case
    return np.nan

c_insens, u_rec_insens = np.NaN # delta-insensitive recovery coefficients

In [ ]:
# TODO: Plot recovery (in the same plot with the original)

### Part 2: Plot the recovery SNR

In [ ]:
# Compute SNR 
# the SNR formula given in the question prompt is for whole-signal SNR
# which is a single value
# for SNR vs t plot use the function one below 
def SNR_f(u, u_rec):
    return 10 * np.log10(u**2 / (u-u_rec)**2)

In [ ]:
# TODO: plot the error and SNR as function of time

### Part 3: Compare the $\delta$-insensitive recovery result with the $\delta$-sensitive result 

In [ ]:
# TODO: implement delta-sensitive case

c_sens, u_rec_sens = np.NaN # delta-sensitive recovery coefficients

In [ ]:
# TODO: Compare between sensitive and insensitive, 
# plot both outputs in same plot with original 
# (make use of legends and labels, feel free to adjust linewidth)

# plot both SNR in the same plot

# Problem 2

### Part 1. Generate and show the inpulse response of the filter
***HINT:*** *For this problem, pay attention to singal time vector, filter time vector and their convolution output lengths.*

In [ ]:
b = 20

In [ ]:
T_1 = 0
T_2 = 0.1
t_filt = np.arange(T_1, T_2, dt)
a = 150
h = (
    3
    * a
    * np.exp(-100 * t_filt)
    * (
        (a * t_filt) ** 3 / np.math.factorial(3)
        - (a * t_filt) ** 5 / np.math.factorial(5)
    )
)  # dendritic filter

In [ ]:
# TODO: plot the inpulse response

### Part 2. Encode a randomly generated input stimulus (from Problem 1) using a reduced PIF neuron that is equivalent to the RHH neuron model. 

In [ ]:
# TODO: filter signal
v = np.NaN

In [ ]:
# TODO: compute iPRC at given bias
rhh = HodgkinHuxley3State()

# Plot the iPRC

In [ ]:
# TODO: Compute PIF from iPRC
pif_spike_time =  np.nan

### Part 3. Recover the signal $u(t)$ from the recieved spikes.

**TODO: A.** Write down the t-transform of the encoding circuit in an inner product form

**TODO: B.** Write down $q_k$ and $[G]_{lk}$, as well the important procedures/steps to obtain $u(t)$ given $h$ (in equation form would suffice)

### Part 4. Recover the signal $u(t)$ from PIF spike times and show the statistics

In [ ]:
# TODO:implement PIF decoder

# def compute_G():
# def compute_q():

In [ ]:
# TODO: recover u(t)
c_rec, u_rec = np.nan

In [ ]:
# TODO: plot recovery result (in same plot as original), and SNR (as fn of time)
# The recovery may be bad at the boundary because all signals are finite, 
# you can focus on the middle part of the signal for comparison.

### Miscellaneous (attention)
* please remember to include your name in the corresponding field at the start of this notebook, otherwize an automatic 0 grade will be received for any missing member
* please submit one copy per team
* please clearly present your solutions and answers with proper use of plots, markdown and comments
* please start the assignment as soon as possible, submission deadlines are strictly followed and late submissions after we posted the solution (ususlly on the same day as the deadline) will receive automatic 0
* please email both TAs at least 48hrs ahead of deadline if you cannot submit an assignment on time (due to exam schedules or  overwhelming deadlines, etc), we are flexible as long as we are informed
* please post your questions on Ed Discussions, and attend the office hours (every Thursday evening); **more on Ed**:
    * while discussing on Ed, please make good use of the reply and comment features inside threads to avoid cluttering with multiple threads regarding the same question from the same user
    * please refrain from asking questions or posting threads that would explicitly share the right answer with everyone
    * only post code snipets when you have narrowed down the issue to that particular section and only share ploted results only when you have good reason to suspect their correctness
    * once you finish debugging your code by the end of a thread, please avoid languages such as "is this correct now?" since we aren't really allowed to explicitly say yes when they are; be confident in your hardwork.